In [93]:
import math
import numpy as np
import torch as t
from torch.distributions import Bernoulli, Normal
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib widget
from ipywidgets import FloatSlider, IntSlider, interact, interact_manual

$$
\newcommand{\bracket}[3]{\left#1 #3 \right#2}
\newcommand{\b}{\bracket{(}{)}}
\newcommand{\Bernoulli}{{\rm Bernoulli}\b}
\newcommand{\x}{\mathbf{x}}
\newcommand{\X}{\mathbf{X}}
\newcommand{\m}{\boldsymbol{\mu}}
\newcommand{\P}{{\rm P}\b}
\newcommand{\dd}[2][]{\frac{\partial #1}{\partial #2}}
\newcommand{\S}{\mathbf{\Sigma}}
\newcommand{\Sh}{\mathbf{\hat{\Sigma}}}
\newcommand{\mh}{\boldsymbol{\hat{\mu}}}
\newcommand{\N}{\mathcal{N}\b}
\newcommand{\det}{\bracket{\lvert}{\rvert}}
\newcommand{\sb}{\bracket{[}{]}}
\newcommand{\E}{\mathbb{E}\sb}
\newcommand{\Var}{{\rm Var}\sb}
\newcommand{\Cov}{{\rm Cov}\sb}
\DeclareMathOperator*{\argmax}{arg\,max}
\DeclareMathOperator*{\argmin}{arg\,min}
\newcommand{\ph}{\hat{p}}
\newcommand{\at}{\bracket{.}{\rvert}}
\newcommand{\w}{\mathbf{w}}
\newcommand{\W}{\mathbf{W}}
\newcommand{\W}{\mathbf{W}}
\newcommand{\Wh}{\mathbf{\hat{W}}}
\newcommand{\Y}{\mathbf{Y}}
\newcommand{\L}{\mathcal{L}}
\newcommand{\wh}{\mathbf{\hat{w}}}
\newcommand{\y}{\mathbf{y}}
\newcommand{\0}{\mathbf{0}}
\newcommand{\I}{\mathbf{I}}
\newcommand{\La}{\mathbf{\Lambda}}
\newcommand{\S}{\mathbf{\Sigma}}
\newcommand{\Sprior}{\S_\text{prior}}
\newcommand{\Spost}{\S_\text{post}}
\newcommand{\mprior}{\m_\text{prior}}
\newcommand{\mpost}{\m_\text{post}}
\newcommand{\Xt}{\tilde{\X}}
\newcommand{\yt}{\tilde{\y}}
$$

<h1> Question sheet 2: classification and clustering </h1>

<h3> Question 1: Classification: maximum likelihood </h3>

Find the best model on the following test-set, using the log-likelihood as the objective.

```  
 x_0    x_1   y
-2.1   -3.2   0
-3.4   -1.2   0
 1.2    3.6   1
-0.1    0.8   1
```
The models are,
1. $l = x_0 + x_1$
2. $l = 4 x_0 + 0.01 x_1$
3. $l = 100 x_0 + 100 x_1$

where $l$ is the logits-value for $y=1$.

In [23]:
X = t.tensor([
    [-2.1, -3.2],
    [-3.4, -1.2],
    [ 1.2,  3.6],
    [-0.1,  0.8]
])

y = t.tensor([
    [0.],
    [0.],
    [1.],
    [1.]
])

W = t.tensor([
    [1, 4, 10],
    [1, 0.01, 10]
])


Compute the logits-value for each datapoint,

In [24]:
l_datapoints = t.zeros(4, 3)
#All datapoints, for first model,
l[0, 0] = 1.0*(-2.1) + 1.0*(-3.2)
l[1, 0] = 1.0*(-3.4) + 1.0*(-1.2)
l[2, 0] = 1.0*( 1.2) + 1.0*( 3.6)
l[3, 0] = 1.0*(-0.1) + 1.0*( 0.8)
 
l[0, 1] = 4.0*(-2.1) + 0.01*(-3.2)
l[1, 1] = 4.0*(-3.4) + 0.01*(-1.2)
l[2, 1] = 4.0*( 1.2) + 0.01*( 3.6)
l[3, 1] = 4.0*(-0.1) + 0.01*( 0.8)

l[0, 2] = 10.0*(-2.1) + 10.0*(-3.2)
l[1, 2] = 10.0*(-3.4) + 10.0*(-1.2)
l[2, 2] = 10.0*( 1.2) + 10.0*( 3.6)
l[3, 2] = 10.0*(-0.1) + 10.0*( 0.8)

assert t.allclose(l, X @ W)

Compute the log-probability for all models, and for $y=1$,

\begin{align}
  \P{y=1| \ell} &= \sigma\b{\ell} = \frac{1}{1+e^{-\ell}}\\
  \log \P{y=1| \ell} &= \log \sigma\b{\ell} = -\log \b{1+e^{-\ell}}.
\end{align}

For $y=0$,

\begin{align}
  \P{y=0| \ell} &= 1 - \sigma\b{\ell}\\
   &= 1 -\frac{1}{1+e^{-\ell}}\\
   &= \frac{1+e^{-\ell} -1}{1+e^{-\ell}}\\
   &= \frac{e^{-\ell}}{1+e^{-\ell}}\\
   &= \frac{1}{1+e^{\ell}}\\
\end{align}




In [44]:
lp = t.zeros(3)
lp[0] = -t.log(1+t.exp( l[0,0])) +\
        -t.log(1+t.exp( l[1,0])) +\
        -t.log(1+t.exp(-l[2,0])) +\
        -t.log(1+t.exp(-l[3,0]))
lp[1] = -t.log(1+t.exp( l[0,1])) +\
        -t.log(1+t.exp( l[1,1])) +\
        -t.log(1+t.exp(-l[2,1])) +\
        -t.log(1+t.exp(-l[3,1]))
lp[2] = -t.log(1+t.exp( l[0,2])) +\
        -t.log(1+t.exp( l[1,2])) +\
        -t.log(1+t.exp(-l[2,2])) +\
        -t.log(1+t.exp(-l[3,2]))
assert t.allclose(lp, Bernoulli(logits=l).log_prob(y).sum(0))
lp


tensor([-4.2636e-01, -9.1636e-01, -9.1142e-04])

The model with the largest log-probability is model 3.  (Note that in the discrete case, probabilities can't be bigger than $1$, so log-probabilities must be smaller than zero).

<h3> Question 2: Classification: maximum a-posteriori </h3>
    
Using the models and test-set given in Q1, use the log-joint probability as the objective to find the best model.  Use the following prior over the weights,

\begin{align}
  \P{\w} &= \N{\w; \0, \I}
\end{align}

Compute the log-prior for each model

\begin{align}
  \log \P{\w} &= -\tfrac{1}{2} \sum_i w_i^2 + \text{const}
\end{align}

In [51]:
log_prior = t.zeros(3)
log_prior[0] = -0.5*(1**2  + 1**2)
log_prior[1] = -0.5*(4**2  + 0.01**2)
log_prior[2] = -0.5*(10**2 + 10**2)

assert t.allclose(log_prior, -0.5*(W**2).sum(0))

log_prior

tensor([  -1.0000,   -8.0000, -100.0000])

Now, compute the log-joint as the sum of the log-prior and log-likelihood,

In [52]:
log_joint = t.zeros(3)
log_joint[0] = log_prior[0] + lp[0]
log_joint[1] = log_prior[1] + lp[1]
log_joint[2] = log_prior[2] + lp[2]

assert t.allclose(log_joint, log_prior + lp)

log_joint

tensor([  -1.4264,   -8.9164, -100.0009])

<h3> Question 3: Classification: test-error </h3>
    
Using the models and test-set given in Q1, use classification error to find the one model that performs worse than the others.  Use a classification boundary of $p=0.5$.

Typically, we'd need to compute the actual probabilities here.  But in this specific example, the threshold is $p=0.5$, which corresponds to a logits-threshold of 0.  So to classify datapoints, we just need to see whether the logits-values are above zero.

In [53]:
l

tensor([[ -5.3000,  -8.4320, -53.0000],
        [ -4.6000, -13.6120, -46.0000],
        [  4.8000,   4.8360,  48.0000],
        [  0.7000,  -0.3920,   7.0000]])

Therefore, the predictions are,

In [55]:
(0 < l).int()

tensor([[0, 0, 0],
        [0, 0, 0],
        [1, 1, 1],
        [1, 0, 1]], dtype=torch.int32)

Compared to the original $y$,

In [56]:
y

tensor([[0.],
        [0.],
        [1.],
        [1.]])

We see that the second (middle) model makes one error.

<h3> Question 4: K-nearest neighbours </h3>

Find the predicted class-label for the test point (0.2, -0.3), with $K=3$.  (Using the standard Euclidean distance; note that the data is the same as above). 
```  
 x_0    x_1   y
-2.1   -3.2   0
-3.4   -1.2   0
-2.6   -2.7   0
 3.2    2.1   1
 1.2    3.6   1
 0.6    0.0   1
```

In [60]:
X = t.tensor([
    [-2.1, -3.2],
    [-3.4, -1.2],
    [-2.6, -2.7],
    [ 3.2,  2.1],
    [ 1.2,  3.6],
    [-0.6,  0.0]
])

y = t.tensor([
    [0.],
    [0.],
    [0.],
    [1.],
    [1.],
    [1.]
])

test_datapoint = t.tensor([0.2, -0.3])

First, compute (squared) distances from the test datapoint, $(0.2, -0.3)$, to the training datapoints,

In [78]:
d2s = t.zeros(6)
d2s[0] = (0.2 - (-2.1))**2 + (-0.3 - (-3.2))**2
d2s[1] = (0.2 - (-3.4))**2 + (-0.3 - (-1.2))**2
d2s[2] = (0.2 - (-2.6))**2 + (-0.3 - (-2.7))**2
d2s[3] = (0.2 - ( 3.2))**2 + (-0.3 - ( 2.1))**2
d2s[4] = (0.2 - ( 1.2))**2 + (-0.3 - ( 3.6))**2
d2s[5] = (0.2 - (-0.6))**2 + (-0.3 - ( 0.0))**2

assert t.allclose(d2s, ((test_datapoint-X)**2).sum(1))

d2s

tensor([13.7000, 13.7700, 13.6000, 14.7600, 16.2100,  0.7300])

Smallest distances are for datapoints 0, 2, 5.

Therefore, the datapoint should be classified as $y=0$.

<h3> Question 5: Weighted-nearest neighbours </h3>

For the dataset and test-point given above, compute the prediction for weighted-nearest neighbours where we use a weighting function of,

\begin{align}
  k(\x, \y) &= \exp\b{-\tfrac{1}{4}\sum_i \b{x_i-y_i}^2}
\end{align}

We can reuse the squared distances computed in the previous section.

In [80]:
ks = t.zeros(6)
ks[0] = (-d2s[0]/4).exp()
ks[1] = (-d2s[1]/4).exp()
ks[2] = (-d2s[2]/4).exp()
ks[3] = (-d2s[3]/4).exp()
ks[4] = (-d2s[4]/4).exp()
ks[5] = (-d2s[5]/4).exp()

assert t.allclose(ks, (-d2s/4).exp())

ks

tensor([0.0325, 0.0320, 0.0334, 0.0250, 0.0174, 0.8332])

Remember in the lectures we had the following formula,

\begin{align}
  p_{\lambda} &= \frac{\sum_{\lambda'}k(\x_\lambda, \x_{\lambda'}) \delta_{1, y_{\lambda'}}}{\sum_{\lambda''} k(\x_\lambda, \x_{\lambda''})}
\end{align}

But as the denominator doesn't depend on the class, we just need the numerator, and this amounts to summing the kernels for datapoints belonging to each class.

In [83]:
sum_class_0 = ks[0] + ks[1] + ks[2]
sum_class_1 = ks[3] + ks[4] + ks[5]
(sum_class_0, sum_class_1)

(tensor(0.0979), tensor(0.8755))

So this weighted-nearest neighbours algorithm classifies the test-point as $y=1$.

<h3> Question 6: Bayesian classification </h3>

Given training data 
```  
 x_0   y
-2.1   0
-3.4   0
-2.6   0
 3.2   1
 1.2   1
 0.6   1
```

Fit Gaussian distributions to each cluster using maximum-likelihood, then compute the corresponding posterior over the class-label for $x_0=-0.3$ (specifically $\P{y=1| x_0=-0.3}$).

Begin by fitting distributions for each class,

In [91]:

#means E[x_0]
m0 = ((-2.1) + (-3.4) + (-2.6))/3
m1 = (( 3.2) + ( 1.2) + ( 0.6))/3

#E[x_0^2]
Ex2_0 = ((-2.1)**2 + (-3.4)**2 + (-2.6)**2)/3
Ex2_1 = (( 3.2)**2 + ( 1.2)**2 + ( 0.6)**2)/3

#variances, Var[x_0]

v0 = Ex2_0 - m0**2
v1 = Ex2_1 - m1**2

#standard deviations

s0 = math.sqrt(v0)
s1 = math.sqrt(v1)

#sanity check
print((m0, s0))
print((m1, s1))

(-2.6999999999999997, 0.5354126134736357)
(1.6666666666666667, 1.1115554667022045)


In [102]:
#probability of test-point under each class distribution, P(x| y)
#test point:
x = -0.3

p_like_0 = math.exp(-(x-m0)**2/(2*v0))/math.sqrt(2*math.pi*v0)
p_like_1 = math.exp(-(x-m1)**2/(2*v1))/math.sqrt(2*math.pi*v1)

assert t.allclose(t.tensor([p_like_0]), Normal(m0, s0).log_prob(x).exp())
assert t.allclose(t.tensor([p_like_1]), Normal(m1, s1).log_prob(x).exp())

(p_like_0, p_like_1)

(3.229065876417563e-05, 0.07502778775431443)

We have just obtained $\P{x| y=0}$ and $\P{x| y=1}$.  To convert this into the posterior, $\P{y=1| x}$, we need to apply Bayes theorem,

\begin{align}
  \P{y| x} &= \frac{\P{x| y} \P{y}}{\P{x}}
\end{align}

But as the denominator doesn't depend on $y$, and the prior is uniform, we have,

\begin{align}
  \P{y| x} &\propto \P{x| y}
\end{align}

Thus, we just need to normalise the probabilities computed above, such that they sum to one,

In [104]:
p_post_1 = p_like_1/(p_like_0+p_like_1)
p_post_1

0.9995698024909264

So, $\P{y=1| x} = 0.9996$.

<h3> Question 7: Classification vs clustering </h3>

Which of these cluster-assignments are equivalent?

```
y_0   y_1   y_2   y_3
 0     1     2     0
 1     0     0     1
 2     2     1     2
 1     0     0     1
 2     2     1     2
 0     1     2     0
```

Ans: all of them are equivalent (because they all group the same datapoints together).

<h3> Question 7: KNN M-step </h3>

For the following data-points, and cluster assignments, $z$, perform one M-step to find the corresponding cluster-centers.

```  
 x_0    x_1   z
-2.1   -3.2   1
-3.4   -1.2   0
-2.6   -2.7   1
 3.2    2.1   0
 1.2    3.6   1
 0.6    0.0   0
```



In [113]:
#mean for cluster 0
m0 = t.zeros(2)
m0[0] = ((-2.1) + (-2.6) + (1.2))/3
m0[1] = ((-3.2) + (-2.7) + (3.6))/3
#mean for cluster 1
m1 = t.zeros(2)
m1[0] = ((-3.4) + ( 3.2) + (-0.6))/3
m1[1] = ((-1.2) + ( 2.1) + ( 0.0))/3
(m0, m1)

(tensor([-1.1667, -0.7667]), tensor([-0.2667,  0.3000]))

<h3> Question 8: KNN E-step </h3>

Given cluster centers in the previous question, update the cluster assignments.

It should be fairly obvious that cluster 0 is closer to datapoints 0, 1, 2, and cluster 1 is closer to datapoints 3 and 4.

For datapoint 5, we should probably do the calculation.

In [119]:
X = t.tensor([
    [-2.1, -3.2],
    [-3.4, -1.2],
    [-2.6, -2.7],
    [ 3.2,  2.1],
    [ 1.2,  3.6],
    [-0.6,  0.0]
])

In [118]:
ds = t.zeros(2)
ds[0] = (X[5, 0] - m0[0])**2 + (X[5, 1] - m0[1])**2
ds[1] = (X[5, 0] - m1[0])**2 + (X[5, 1] - m1[1])**2
ds

tensor([0.9089, 0.2011])

So the fifth datapoint should be assigned to cluster 0.